In [7]:
import os, sys, shutil, zipfile, csv
import requests
import psycopg2
import fiona
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
from collections import defaultdict
from datetime import time

from sqlalchemy import URL, create_engine, text
from trino.auth import OAuth2Authentication

In [8]:
# Add in parent directories to sys.path to get multiHook library
current_dir = os.getcwd()
for x in range(4):  # Look four levels up
    parent_dir = os.path.dirname(current_dir)
    if parent_dir not in sys.path:
        sys.path.append(parent_dir)
    current_dir = parent_dir

import multihook.pycnxn.dbhook as dbhook

In [9]:
def import_bundle(
    bundle,
    import_trips=True,
    import_shapes=True,
    import_stops=True,
    import_stoptimes=True,
    import_calendar=True,
    import_calendar_dates=True,
    import_routes=True,
    cache_bundle=False,
):
    """Import GTFS data from the following tables in ADLS:
    - core.dbo.fact_gtfs_trips
    - core.dbo.fact_gtfs_shapes
    - core.dbo.fact_gtfs_stops
    - core.dbo.fact_gtfs_shape_reference
    - core.dbo.fact_gtfs_stop_times

    - ADD IN CALENDAR, CALENDAR DATES, AND ROUTES
    """

    # GTFS bundle can take some time to import. If there is already a cached GTFS bundle pickle file in the repository, read in that data instead
    picklefile = bundle + ".pickle"
    if os.path.isfile(picklefile) == True:
        with open(picklefile, "rb") as handle:
            gtfs_bundle = pickle.load(handle)
            return gtfs_bundle

    con = create_engine(
        r"trino://trino-route-trino.apps.mtasiprod.eastus.aroapp.io:443/mtadatalake",
        connect_args={
            "auth": OAuth2Authentication(),
            "http_scheme": "https",
        }
    )
    cur = con.connect()
    
    output = {}
    # Import trips
    trip_sql = f"""
    with agency as ( -- get agency id for each route
        select distinct route_id, agency_id from mtadatalake.core.fact_gtfs_routes 
        where bundle = '{bundle}'
        )
    SELECT trips.route_id ,trip_id, service_id,trip_headsign,direction_id,block_id,shape_id,boarding_type,bundle, agency.agency_id
    FROM mtadatalake.core.fact_gtfs_trips trips
    join agency on trips.route_id = agency.route_id
    where trips.bundle = '{bundle}'
    """
    if import_trips == True:
        print('Loading trips')
        f = cur.execute(text(trip_sql))
        trips_df = pd.DataFrame(f.fetchall())
        output["trips"] = trips_df

    # Import shapes
    shape_sql = f"""
    SELECT shape_id, shape_pt_sequence, shape_pt_lat, shape_pt_lon, bundle
    FROM mtadatalake.core.fact_gtfs_shapes
    where bundle = '{bundle}'
    """
    if import_shapes == True:
        print("Loading shapes")
        f = cur.execute(text(shape_sql))
        shapes_df = pd.DataFrame(f.fetchall())
        output["shapes"] = shapes_df

    # Import stops
    stop_sql = f"""
    SELECT fact_gtfs_stops.stop_id, stop_name, stop_lat, stop_lon, shape_ref.revenue_stop, bundle
    FROM mtadatalake.core.fact_gtfs_stops
    left join (
        SELECT stop_id, MAX(revenue_stop) AS revenue_stop
        FROM mtadatalake.core.fact_gtfs_shape_reference
        where bundle = '{bundle}'
        GROUP BY stop_id
    ) shape_ref
    on fact_gtfs_stops.stop_id = shape_ref.stop_id
    where bundle = '{bundle}'
    """
    if import_stops == True:
        print("Loading stops")
        f = cur.execute(text(stop_sql))
        stops_df = pd.DataFrame(f.fetchall())
        output["stops"] = stops_df

    # Import stop_times
    stoptime_sql = f"""
    SELECT trip_id, stop_id, arrival_time, departure_time, timepoint, stop_sequence, pickup_type, drop_off_type, bundle
    FROM mtadatalake.core.fact_gtfs_stop_times
    where bundle = '{bundle}'
    """
    if import_stoptimes == True:
        print("Loading stop times")
        f = cur.execute(text(stoptime_sql))
        stoptimes_df = pd.DataFrame(f.fetchall())
        output["stoptimes"] = stoptimes_df

    # import calendar
    calendar_sql = f""" 
    SELECT service_id, monday, tuesday, wednesday, thursday, friday, saturday, sunday, start_date, end_date, bundle, modified_time, loaded_time
    FROM mtadatalake.core.fact_gtfs_calendar
    where bundle = '{bundle}'
    """
    if import_calendar == True:
        print("Loading calendar")
        f = cur.execute(text(calendar_sql))
        calendar_df = pd.DataFrame(f.fetchall())
        output["calendar"] = calendar_df

    #import calendar_dates
    calendar_dates_sql = f"""
    SELECT service_id, "date", exception_type
    FROM mtadatalake.core.fact_gtfs_calendar_dates
    where bundle = '{bundle}'
    """
    if import_calendar_dates == True:
        print("Loading calendar_dates")
        f = cur.execute(text(calendar_dates_sql))
        calendar_dates_df = pd.DataFrame(f.fetchall())
        output["calendar_dates"] = calendar_dates_df

    #import routes
    routes_sql = f"""
    SELECT route_id, route_short_name, route_long_name
    FROM mtadatalake.core.fact_gtfs_routes
    where bundle = '{bundle}'
    """
    if import_calendar_dates == True:
        print("Loading routes")
        f = cur.execute(text(routes_sql))
        routes_df = pd.DataFrame(f.fetchall())
        output["routes"] = routes_df

    
    if cache_bundle == True:
        with open(picklefile, "wb") as handle:
            pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"GTFS data for {bundle} successfully loaded and cached")
    else:
       print(f"GTFS data for {bundle} successfully loaded") 

    return output

In [ ]:
#### Instead of having a variant class that generates a schedule each time, 

### Variant class with metrics as attributes

In [10]:
class Variant:
    def __init__(self, shape_id, bundle):
        self.shape_id = shape_id
        self.bundle = bundle

        ### GTFS 
        self.stops = self.bundle['stops']
        self.stop_times = self.bundle['stoptimes']
        self.trips = self.bundle['trips']
        self.calendar = self.bundle['calendar']
        self.calendar_dates = self.bundle['calendar_dates']
        
        

        ### Metrics
        self.trip_count = self._get_trip_count()
        if self.trip_count == 0: # to filter out variants with no trips later
            raise ValueError
        self.stop_set = self._get_stop_set()
        self.stop_count = self._get_stop_count()
        self.headsign = self._get_headsign()
        self.startend = self._get_start_end_stops()
        self.geometry = self._get_geometry()
        self.length = self._get_shape_length()
        self.direction = self._get_direction()
        self.daytype = self._get_daytype()
        self.school = self._get_runs_only_during_school_hours()

    ### Metric 1 Getter ### 
    def _get_trip_count(self):
        return self.trips[self.trips['shape_id'] == self.shape_id].shape[0]

    ### Metric 2 Getter ### 
    def _get_stop_set(self):
        # Create mapping: trip_id → shape_id
        trip_to_shape = self.trips.set_index('trip_id')['shape_id'].to_dict()
    
        # Find first trip_id for this shape_id
        shape_to_trip = {}
        for trip_id, shape_id in trip_to_shape.items():
            if shape_id not in shape_to_trip:
                shape_to_trip[shape_id] = trip_id
    
        shape_id = self.shape_id
        if shape_id not in shape_to_trip:
            return []
    
        trip_id = shape_to_trip[shape_id]
    
        # Get stop_times for this trip
        grouped = self.stop_times.groupby('trip_id')
        if trip_id not in grouped.groups:
            return []
    
        trip_stops = grouped.get_group(trip_id).sort_values('stop_sequence')
        if trip_stops.empty:
            return []
    
        # Map stop_id → stop_name
        stop_id_to_name = self.stops.set_index('stop_id')['stop_name'].to_dict()
    
        # Ordered list of stop names
        ordered_stop_names = [
            stop_id_to_name.get(row['stop_id'], row['stop_id'])
            for _, row in trip_stops.iterrows()
        ]
        return ordered_stop_names

    ### Metric 3 ###
    def _get_stop_count(self):
        return len(self._get_stop_set())

        
    ### Metric 4 Getter ###
    def _get_headsign(self):
        headsign_map = self.trips[['shape_id', 'trip_headsign']].drop_duplicates(subset='shape_id')
        headsign_dict = dict(zip(headsign_map['shape_id'], headsign_map['trip_headsign']))
        return headsign_dict.get(self.shape_id, None)
    
    ### Metric 5 Getter ### 
    def _get_start_end_stops(self):
        stop_names = self._get_stop_set()
        if not stop_names:
            return (None, None)
        return (stop_names[0], stop_names[-1])
        
    ### Metric 6: Geometry Getter ###
    def _get_geometry(self):
        # Filter trips for this shape
        relevant_trips = self.trips[self.trips['shape_id'] == self.shape_id]
        if relevant_trips.empty:
            return None
    
        # Get stop_times for first trip in shape
        relevant_stop_times = self.stop_times[self.stop_times['trip_id'].isin(relevant_trips['trip_id'])]
        first_trip_id = relevant_trips.iloc[0]['trip_id']
        trip_stop_times = relevant_stop_times[relevant_stop_times['trip_id'] == first_trip_id]
        trip_stop_times = trip_stop_times.sort_values(by='stop_sequence')
    
        # Merge with stops to get coordinates
        stops_with_coords = trip_stop_times.merge(self.stops, on='stop_id', how='left')
        if stops_with_coords[['stop_lat', 'stop_lon']].isnull().any().any():
            return None  # Missing coordinate data
    
        # Create LineString
        return LineString(zip(stops_with_coords['stop_lon'], stops_with_coords['stop_lat']))

    ### Metric 7 Getter ###
    def _get_shape_length(self):
        geom = self._get_geometry()
        if geom is None:
            return np.nan
    
        geo_line = gpd.GeoSeries([geom], crs="EPSG:4326").to_crs("EPSG:2263")
        return geo_line.length.iloc[0]

        
    ### Metric 8 ### 
    def _get_direction(self):
        directions = self.trips[self.trips['shape_id'] == self.shape_id]['direction_id'].drop_duplicates()
        if directions.empty:
            return None
        return directions.iloc[0]


    ### Metric 9 ### 
    def _get_daytype(self):
        shape_trips = self.trips[self.trips['shape_id'] == self.shape_id]
        service_ids = shape_trips['service_id'].unique()
        service_days = self.calendar[self.calendar['service_id'].isin(service_ids)]
    
        # List of day columns
        day_columns = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    
        # Sum across all service_ids to find which daytypes are served
        active_days = (service_days[day_columns].sum() > 0)
    
        # Get list of active day names
        active_day_names = active_days[active_days].index.tolist()
    
        # Decide daytype based on active days
        weekdays = {'monday', 'tuesday', 'wednesday', 'thursday', 'friday'}
        weekends = {'saturday', 'sunday'}
    
        active_day_set = set(active_day_names)
    
        if active_day_set.issubset(weekdays):
            return 'weekday'
        elif active_day_set.issubset(weekends):
            return 'weekend'
        else:
            return 'weekday'
            
    ### Metric 10 ### 
    def _get_runs_only_during_school_hours(self):
        morning_start = time(7, 0)
        morning_end = time(9, 0)
        afternoon_start = time(14, 0)
        afternoon_end = time(16, 0)
    
        variant_trips = self.trips[self.trips['shape_id'] == self.shape_id]
    
        if variant_trips.empty:
            return False
    
        first_stop_times = (
            self.stop_times[
                (self.stop_times['trip_id'].isin(variant_trips['trip_id'])) &
                (self.stop_times['stop_sequence'] == 1)
            ][['trip_id', 'departure_time']]
        )
    
        def parse_time(t_str):
            h, m, s = map(int, t_str.split(':'))
            h = h % 24  # Normalize 24+ hour times
            return time(h, m, s)
    
        first_stop_times['dep_time_obj'] = first_stop_times['departure_time'].apply(parse_time)
    
        def in_school_hours(t):
            return (morning_start <= t <= morning_end) or (afternoon_start <= t <= afternoon_end)
    
        # Check if ALL trips are within school hours
        all_in_school = first_stop_times['dep_time_obj'].apply(in_school_hours).all()
    
        return all_in_school


# Variant List
#### Create a list of all variants for all routes of any bundle. Eventually will compare variants of one bundle to variants of another. 

## Step 1: Loading in Bundles

In [5]:
bundle_names = [
    '2024Jan_Prod_r01_b03_Predate_Shuttles_v2_i1_scheduled',
    '2024April_Prod_r01_b07_Predate_02_Shuttles_2_SCHEDULED'
]
bundle_dfs = [import_bundle(name) for name in bundle_names]

Loading trips
Loading shapes
Loading stops
Loading stop times
Loading calendar
Loading calendar_dates
Loading routes
GTFS data for 2024Jan_Prod_r01_b03_Predate_Shuttles_v2_i1_scheduled successfully loaded
Loading trips
Loading shapes
Loading stops
Loading stop times
Loading calendar
Loading calendar_dates
Loading routes
GTFS data for 2024April_Prod_r01_b07_Predate_02_Shuttles_2_SCHEDULED successfully loaded


#### Step 1a: Filter to most representative days


In [11]:
### Takes in a bundle dictionary, along with the dates and days of week you'd like to filter to,
### and returns the filtered dataframes as the new bundle dict
def bundlefilter(bundle, target_dates, days_of_week):
    trips = bundle['trips']
    calendar = bundle['calendar']
    calendar_dates = bundle['calendar_dates']
    routes = bundle['routes']

    filtered_trips = pd.DataFrame()

    for date, day in zip(target_dates, days_of_week):
        # Base services active on the day
        base_services = calendar[
            (calendar[day] == 1) &
            (calendar['start_date'] <= date) &
            (calendar['end_date'] >= date)
        ]['service_id']

        # Exceptions on that date
        exceptions = calendar_dates[calendar_dates['date'] == date]
        removed = exceptions[exceptions['exception_type'] == 2]['service_id']
        added = exceptions[exceptions['exception_type'] == 1]['service_id']

        # Apply exception logic
        final_services = pd.concat([
            base_services[~base_services.isin(removed)],
            added
        ]).drop_duplicates()

        # Filter trips for this date
        trips_today = trips[trips['service_id'].isin(final_services)]
        filtered_trips = pd.concat([filtered_trips, trips_today])

    # Remove duplicates
    filtered_trips = filtered_trips.drop_duplicates()

    # Filter stop_times to match trips
    stop_times = bundle['stoptimes']
    filtered_stop_times = stop_times[stop_times['trip_id'].isin(filtered_trips['trip_id'])]

    # Return a new filtered bundle
    return {
        'trips': filtered_trips.reset_index(drop=True),
        'stoptimes': filtered_stop_times.reset_index(drop=True),
        'stops': bundle['stops'], 
        'shapes': bundle['shapes'],  
        'calendar': calendar,
        'calendar_dates': calendar_dates,
        'routes': routes
    }


In [12]:
bundle1 = bundlefilter(bundle_dfs[0],
                                target_dates=[20240108, 20240113, 20240107],
                                days_of_week=['monday', 'saturday', 'sunday'])

bundle2 = bundlefilter(bundle_dfs[1],
                                target_dates=[20240402, 20240406, 20240407],
                                days_of_week=['tuesday', 'saturday', 'sunday'])

##### Comparisons v1

In [13]:
def find_variants(trips, route_id):
    shape_ids = trips[trips['route_id'] == route_id]['shape_id'].dropna().unique()
    return list(shape_ids)

In [14]:
q37_variants_bundle1 = find_variants(bundle1['trips'], 'Q37')
print(q37_variants_bundle1)
q54_variants_bundle1 = find_variants(bundle1['trips'], 'Q54')
print(q54_variants_bundle1)
q10_variants_bundle1 = find_variants(bundle1['trips'], 'Q10')
print(q10_variants_bundle1)


['Q370222', 'Q370223', 'Q370221', 'Q370220', 'Q370158']
['Q54O0031', 'Q54O0030', 'Q54O0032', 'Q54O0257', 'Q54O0259', 'Q54O0255', 'Q54O0258', 'Q54O0252', 'Q54O0262', 'Q540054', 'Q540035', 'Q540227', 'Q540051', 'Q540055', 'Q540050']
['Q100593', 'Q100580', 'Q100521', 'Q100523', 'Q100577', 'Q100578', 'Q100594', 'Q100529']


In [21]:
q37_variants_bundle2 = find_variants(bundle2['trips'], 'Q37')
print(q37_variants_bundle2)
q54_variants_bundle2 = find_variants(bundle2['trips'], 'Q54')
print(q54_variants_bundle2)
q10_variants_bundle2 = find_variants(bundle2['trips'], 'Q10')
print(q10_variants_bundle2)


['Q370222', 'Q370223', 'Q370221', 'Q370220', 'Q370158']
['Q54O0255', 'Q54O0031', 'Q54O0030', 'Q54O0032', 'Q54O0252', 'Q54O0259', 'Q54O0262', 'Q54O0258', 'Q54O0257', 'Q540227', 'Q540035', 'Q540054', 'Q540051', 'Q540055', 'Q540050']
['Q100578', 'Q100593', 'Q100577', 'Q100580', 'Q100594', 'Q100521', 'Q100529', 'Q100523']


#### Creating variant objects, average loading time of 1 minute per route

In [20]:
variants1 = {}
for shape in q37_variants_bundle1:
    try:
        v = Variant(shape, bundle1)
        variants1[shape] = v
    except ValueError:
        # Skip shapes with zero trips
        continue
variants2 = {}
for shape in q54_variants_bundle1:
    try:
        v = Variant(shape, bundle1)
        variants2[shape] = v
    except ValueError:
        # Skip shapes with zero trips
        continue
variants3 = {}
for shape in q10_variants_bundle1:
    try:
        v = Variant(shape, bundle1)
        variants3[shape] = v
    except ValueError:
        # Skip shapes with zero trips
        continue

In [22]:
variants1b = {}
for shape in q37_variants_bundle2:
    try:
        v = Variant(shape, bundle2)
        variants1b[shape] = v
    except ValueError:
        # Skip shapes with zero trips
        continue
variants2b = {}
for shape in q54_variants_bundle2:
    try:
        v = Variant(shape, bundle2)
        variants2b[shape] = v
    except ValueError:
        # Skip shapes with zero trips
        continue
variants3b = {}
for shape in q10_variants_bundle2:
    try:
        v = Variant(shape, bundle2)
        variants3b[shape] = v
    except ValueError:
        # Skip shapes with zero trips
        continue

#### Output: Variant List with inferred Variant Type

In [23]:
def group_variants(variants):
    group = defaultdict(list)
    for v in variants.values():  
        # Create a single string key for easy reading
        key = f"{v.daytype}_{v.direction}"
        group[key].append(v)
    return group
    

In [24]:
def find_main_variant(variants):
    trip_weight = 0.6
    length_weight = 0.3
    stop_weight = 0.1

    def score(v):
        length = v.length if v.length is not None else 0
        return (trip_weight * v.trip_count) + (length_weight * length / 1000) + (stop_weight * v.stop_count)

    return max(variants, key=score)


In [25]:
### Takes in a variant and its corresponding 'main' variant to compare it to
### returns its inferred variant type
def inferred_variant_type(variant, main):
    if variant.shape_id == main.shape_id:
        return "main"
    elif variant.trip_count <= 2: ## If only 1 or 2 trips, classify as other.
        return "other/unknown"
    elif ((main.stop_count - variant.stop_count)/main.stop_count > 0.4) or any(word in variant.headsign.lower() for word in ("limited","ltd")): 
        ## if 'limited' or 'express' in headsign, or if both stop count is at least 40 percent smaller than main
        return "limited"
    elif (variant.daytype == 'weekday') and (variant.school == True):
        ## if on weekdays and durings school hours
        return "school"
    elif ((main.stop_count - variant.stop_count)/main.stop_count) > 0.15 and ((main.length - variant.length)/main.length) > 0.15:
        ## if both stop count and length are more than 15 percent shorter/smaller than main
        return "short"
    else:
        return "branch"

In [26]:
def build_variant_table(variants):
    grouped = group_variants(variants)
    rows = []
    
    for group_key, group in grouped.items():
        main_variant = find_main_variant(group)
        for v in group:
            variant_type = inferred_variant_type(v, main_variant)
            rows.append({
                "shape_id": v.shape_id,
                "daytype_direction": group_key,
                "trip_count": v.trip_count,
                "stop_count": v.stop_count,
                "length": v.length,
                "headsign": v.headsign,
                "start_end_stops": v.startend,
                "inferred_variant_type": variant_type
            })

    
    return pd.DataFrame(rows)

In [27]:
all3 = {'q37':variants1,'q54':variants2,'q10':variants3}
all3b = {'q37':variants1b,'q54':variants2b,'q10':variants3b}

In [28]:
def build_all_routes_variant_table(all_variants_dict):
    dfs = []
    for route_id, variants_dict in all_variants_dict.items():
        df = build_variant_table(variants_dict) 
        df['route_id'] = route_id  # add route id column
        dfs.append(df)
    all_variants_df = pd.concat(dfs, ignore_index=True)
    return all_variants_df
final = build_all_routes_variant_table(all3)
final.to_excel('test.xlsx',index=False)

In [ ]:
# helpers: similarity metrics
def stop_set_similarity(s1, s2):
    if not s1 and not s2:
        return 1.0
    if not s1 or not s2:
        return 0.0
    inter = len(s1 & s2)
    union = len(s1 | s2)
    return inter / union if union else 0.0

def shape_similarity(sh1, sh2):
    if sh1 is None or sh2 is None:
        return 0.0

    # Ensure both are shapely geometries
    if not hasattr(sh1, 'intersection'):
        sh1 = LineString(sh1)
    if not hasattr(sh2, 'intersection'):
        sh2 = LineString(sh2)

    # Use intersection length vs union length
    inter_length = sh1.intersection(sh2).length
    union_geom = unary_union([sh1, sh2])
    union_length = union_geom.length

    return inter_length / union_length if union_length else 0.0
    
def start_end_match(a, b):
    return 1.0 if (a and b and a[0] == b[0] and a[1] == b[1]) else 0.0

def headsign_match(a, b):
    if a is None or b is None:
        return 0.0
    return 1.0 if a.strip().lower() == b.strip().lower() else 0.0

def ratio_score(a, b):
    # returns value in [0,1] closeness of ratio to 1
    if a is None or b is None or (a == 0 and b == 0):
        return 0.0
    r = min(a,b) / max(a,b) if max(a,b) else 0.0
    return r

# composite similarity (weights can be changed)
def variant_similarity(v1, v2, weights=None):
    if weights is None:
        weights = {
            'stop_set_similarity': 0.15,
            'shape_similarity': 0.20,
            'startend': 0.20,
            'headsign': 0.10,
            'trip_ratio': 0.10,
            'length_ratio': 0.15,
            'stopcount_ratio': 0.10
        }
    sc = {}
    sc['stop_set_similarity'] = stop_set_similarity(v1['stop_id_set'], v2['stop_id_set'])
    sc['shape_similarity'] = shape_similarity(v1.get('geometry'), v2.get('geometry'))
    sc['startend'] = start_end_match(v1.get('startend'), v2.get('startend'))
    sc['headsign'] = headsign_match(v1.get('headsign'), v2.get('headsign'))
    sc['trip_ratio'] = ratio_score(v1.get('trip_count',0), v2.get('trip_count',0))
    sc['length_ratio'] = ratio_score(v1.get('length',0), v2.get('length',0))
    sc['stopcount_ratio'] = ratio_score(v1.get('stop_count',0), v2.get('stop_count',0))

    total = 0.0
    for k,w in weights.items():
        total += w * sc[k]
    return total, sc  # returns composite and breakdown
